In [1]:
document = """To Sherlock Holmes she is always the woman. I have
seldom heard him mention her under any other name. In his eyes she
eclipses and predominates the whole of her sex. It was not that he
felt any emotion akin to love for Irene Adler. All emotions, and that
one particularly, were abhorrent to his cold, precise but admirably
balanced mind. He was, I take it, the most perfect reasoning and
observing machine that the world has seen, but as a lover he would
have placed himself in a false position. He never spoke of the softer
passions, save with a gibe and a sneer. They were admirable things for
the observer-excellent for drawing the veil from men’s motives and
actions. But for the trained reasoner to admit such intrusions into
his own delicate and finely adjusted temperament was to introduce a
distracting factor which might throw a doubt upon all his mental
results. Grit in a sensitive instrument, or a crack in one of his own
high-power lenses, would not be more disturbing than a strong emotion
in a nature such as his. And yet there was but one woman to him, and
that woman was the late Irene Adler, of dubious and questionable
memory.
"""

In [2]:
import nltk
from nltk.tokenize.punkt import PunktSentenceTokenizer

sentence_tokenizer = PunktSentenceTokenizer()
sentences_original = sentence_tokenizer.tokenize(" ".join(document.strip().split("\n")))

from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

document = document.lower().split()
document = [w for w in document if not w in stops]

document = " ".join(document)

document = " ".join(document.strip().split("\n"))

sentences = sentence_tokenizer.tokenize(document)

import re

for i in xrange(0, len(sentences)) :
    sentences[i] = re.sub("[^a-zA-Z]", " ", sentences[i])

In [3]:
from gensim.models import Word2Vec
model = Word2Vec.load("~/Desktop/PClub_DTC/NLP_Learning/Kaggle_BagOfWords/300features_40minwords_10context")

index2word_set = set(model.wv.vocab)

In [4]:
import numpy as np

sentence_matrix = []
sentence_length = []

num_features = 300

for sentence in sentences :
    num_words = 0
    words = sentence.split()
    matrix = np.empty(shape=(len(words), 300), dtype=float)
    for word in words : 
        if word in index2word_set :
            matrix[num_words] = model[word]
            num_words = num_words + 1

    extra_rows = len(words) - num_words
    
    for i in xrange(0, extra_rows) :
        matrix = np.delete(matrix, -1, 0)
        
    sentence_matrix.append(matrix)
    sentence_length.append(num_words)
    
similarity_matrix_wv = np.identity(len(sentence_matrix))

for i in xrange(0, len(sentence_matrix)) :
    for j in xrange(i+1, len(sentence_matrix)) :
        cum_sum = (np.dot(sentence_matrix[0], sentence_matrix[1].T)).sum(axis=0).sum()
        cum_sum = (2*cum_sum)/(sentence_length[i] + sentence_length[j])
        similarity_matrix_wv[i][j] = cum_sum
        similarity_matrix_wv[j][i] = cum_sum

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words = 'english')
bow_matrix = vectorizer.fit_transform(sentences_original)

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
transformed_matrix = tfidf_transformer.fit_transform(bow_matrix)

similarity_matrix_ti = transformed_matrix * transformed_matrix.T
similarity_matrix_ti = similarity_matrix_ti.toarray()

In [6]:
similarity_matrix = (similarity_matrix_wv + similarity_matrix_ti)/2

import networkx as nx
nx_graph = nx.from_numpy_matrix(similarity_matrix)
scores = nx.pagerank(nx_graph)

In [7]:
ranked = sorted(((scores[i],i) for (i, s) in enumerate(sentences_original)), reverse=True)

summary_length = len(sentences)/3
ordered_summary_indices = sorted(ranked[i][1] for i in xrange(0, summary_length))

for i in ordered_summary_indices :
    print sentences_original[i]

To Sherlock Holmes she is always the woman.
It was not that he felt any emotion akin to love for Irene Adler.
And yet there was but one woman to him, and that woman was the late Irene Adler, of dubious and questionable memory.


In [8]:
document_reconstructed = " ".join(sentences_original)

from gensim.summarization import summarize
print summarize(document_reconstructed)

It was not that he felt any emotion akin to love for Irene Adler.
All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind.
